In [23]:
import serial

In [24]:
# sendSerialCommand: function for sending commands to the OPS-241B module
def send_serial_cmd(print_prefix, command) :
    data_for_send_str = command
    data_for_send_bytes = str.encode(data_for_send_str)
    print(print_prefix, command)
    ser.write(data_for_send_bytes)
    # Initialize message verify checking
    ser_message_start = '{'
    ser_write_verify = False
    # Print out module response to command string
    while not ser_write_verify :
        data_rx_bytes = ser.readline()
        data_rx_length = len(data_rx_bytes)
        if (data_rx_length != 0) :
            data_rx_str = str(data_rx_bytes)
            if data_rx_str.find(ser_message_start) > 0:
                print(data_rx_str)
                ser_write_verify = True

In [25]:
# Initialize the USB port to read from the OPS-241B module
ser=serial.Serial(
    port = '/dev/cu.usbmodemFA131',
    baudrate = 115200,
    parity = serial.PARITY_NONE,
    stopbits = serial.STOPBITS_ONE,
    bytesize = serial.EIGHTBITS,
    timeout = 1,
    writeTimeout = 2
)
ser.flushInput()
ser.flushOutput()

In [26]:
# Initialize and query Ops241C Module
print("Initializing Ops241C Module\n")
#send_serial_cmd("Module Information: ", '??')
#print('\n')
send_serial_cmd("Set Output Units OFF: ", 'ou')

Initializing Ops241C Module

Set Output Units OFF:  ou
b'{"OutputFeature":"u"}\r\n'


In [27]:
# To wrap in a function
def get_radar_data():
    buffer_string = b''
    while True:
        buffer_string = buffer_string + ser.read(ser.inWaiting())
        #print(buffer_string)
        #exit()
        if b'\r\n' in buffer_string:
            lines = buffer_string.split(b'\r\n') # Guaranteed to have at least 2 entries
            last_received = lines[-2]
            #If the Arduino sends lots of empty lines, you'll lose the
            #last filled line, so you could make the above statement conditional
            #like so: if lines[-2]: last_received = lines[-2]
            buffer_string = lines[-1]
            yield float(last_received)

In [28]:
%matplotlib widget
import numpy as np
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import matplotlib.animation as animation


class Scope(object):
    def __init__(self, ax, maxt=40, dt=0.02):
        self.ax = ax
        self.dt = dt
        self.maxt = maxt
        self.tdata = [0]
        self.ydata = [0]
        self.line = Line2D(self.tdata, self.ydata)
        self.ax.add_line(self.line)
        self.ax.set_ylim(0, 15)
        self.ax.set_xlim(0, self.maxt)

    def update(self, y):
        lastt = self.tdata[-1]
        if lastt > self.tdata[0] + self.maxt:  # reset the arrays
            self.tdata = [self.tdata[-1]]
            self.ydata = [self.ydata[-1]]
            self.ax.set_xlim(self.tdata[0], self.tdata[0] + self.maxt)
            self.ax.figure.canvas.draw()

        t = self.tdata[-1] + self.dt
        self.tdata.append(t)
        self.ydata.append(y)
        self.line.set_data(self.tdata, self.ydata)
        return self.line,


fig, ax = plt.subplots()
scope = Scope(ax)

# pass a generator in "emitter" to produce data for the update func
ani = animation.FuncAnimation(fig, scope.update, get_radar_data, interval=100,
                              blit=True)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …